<a href="https://colab.research.google.com/github/lehuy1912/auto---drive/blob/main/delete_file_duplicate_on_drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import re
import hashlib
from google.colab import drive

# Gắn Google Drive
drive.mount('/content/drive')

# Thư mục cần kiểm tra
folder_path = "/content/drive/MyDrive/PC"  # đổi thành thư mục của bạn

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def normalize_filename(name):
    """
    Chuẩn hóa tên file: bỏ hậu tố _1, _2, (1), (2), ...
    Ví dụ: book.pdf, book_1.pdf, book(2).pdf -> book.pdf
    """
    name = re.sub(r'(_\d+|\(\d+\))(?=\.[^.]+$)', '', name)
    return name.lower()
def file_hash(filepath):
    """Tính hash MD5 để so sánh nội dung file"""
    hash_md5 = hashlib.md5()
    with open(filepath, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()


In [7]:
from tqdm import tqdm

In [9]:
# Bỏ qua file Google shortcut
ignore_ext = {".gsheet", ".gdoc", ".gslides"}

# Đếm tổng số file trước
all_files = []
for root, _, files in os.walk(folder_path):
    for filename in files:
        ext = os.path.splitext(filename)[1].lower()
        if ext not in ignore_ext:
            all_files.append(os.path.join(root, filename))



In [10]:
duplicates = {}

# Duyệt file kèm progress bar
for filepath in tqdm(all_files, desc="Đang kiểm tra", unit="file"):
    filename = os.path.basename(filepath)
    norm_name = normalize_filename(filename)

    try:
        file_md5 = file_hash(filepath)
    except OSError as e:
        print(f"Bỏ qua {filepath}: {e}")
        continue

    key = (norm_name, file_md5)
    if key not in duplicates:
        duplicates[key] = [filepath]
    else:
        duplicates[key].append(filepath)



Đang kiểm tra: 100%|██████████| 3663/3663 [29:55<00:00,  2.04file/s]


In [ ]:
# In kết quả
print("\n==== Kết quả phát hiện file trùng ====\n")
for key, paths in duplicates.items():
    if len(paths) > 1:
        print("Phát hiện file trùng:")
        for p in paths:
            print("  ", p)
        print("-" * 50)

In [ ]:
# Xóa file trùng (giữ lại file đầu tiên)
deleted_files = []
for key, paths in duplicates.items():
    if len(paths) > 1:
        # Giữ lại file đầu tiên
        keep = paths[0]
        for p in paths[1:]:
            try:
                if os.path.exists(p):
                    os.remove(p)
                    deleted_files.append(p)
                else:
                    print(f"Bỏ qua {p}: file không tồn tại")
            except Exception as e:
                print(f"Lỗi khi xóa {p}: {e}")
                continue  # bỏ qua và tiếp tục file khác

# Thống kê cuối cùng
total_files = len(all_files)
duplicate_files = len(deleted_files)
percent_dup = (duplicate_files / total_files) * 100 if total_files > 0 else 0

print("\n==== KẾT QUẢ ====")
print(f"Tổng số file ban đầu : {total_files}")
print(f"Số file đã xóa       : {duplicate_files}")
print(f"Tỷ lệ file trùng     : {percent_dup:.2f}%")

if deleted_files:
    print("\nCác file đã bị xóa:")
    for f in deleted_files:
        print("  ", f)
else:
    print("\nKhông có file trùng để xóa.")
